## Mediation: Psychological Need Satisfaction → Epistemic Trust → Global AI Acceptance

We next test whether global psychological need satisfaction (TENS_Life_mean) is associated with
overall acceptance of AI mental-health technologies (UTAUT_AI_mean) indirectly through epistemic
trust in mental health technologies (ET_mean), while accounting for demographics, symptoms, stigma,
general AI attitudes, culture, and sample type.

Conceptually, this section evaluates a mediation pathway:

- **Predictor (X):** TENS_Life_mean_c – centered Self-Determination (need satisfaction in life)
- **Mediator (M):** ET_mean_c – centered epistemic trust in mental health technologies
- **Outcome (Y):** UTAUT_AI_mean – global AI acceptance for mental health interventions

We estimate:

1. **Path a:** X → M (Does need satisfaction relate to epistemic trust, net of covariates?)
2. **Paths b and c′:** M and X → Y (Does epistemic trust predict AI acceptance after accounting
   for need satisfaction, and is the direct effect of need satisfaction on AI acceptance still
   non-zero?)
3. **Indirect effect (a × b):** The product of a and b, with a non-parametric bootstrap to obtain a
   confidence interval for the mediated effect.

All predictors are either **mean-centered** (continuous variables) or entered as categorical
contrasts (gender, Country, role_label). We use listwise deletion within the mediation model.


# 0.0. Library and Path Setup

In [ ]:
from __future__ import annotations

import warnings
from pathlib import Path
from typing import Dict, List

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf

warnings.filterwarnings("ignore", category=FutureWarning)

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams["axes.titlesize"] = 13
plt.rcParams["axes.labelsize"] = 12
plt.rcParams["font.size"] = 11

PROJECT_ROOT = Path.cwd().resolve()
DATA_DIR = PROJECT_ROOT / "data"
OUTPUT_DIR = DATA_DIR / "output"
PROCESSED_PATH = OUTPUT_DIR / "processed.csv"

processed = pd.read_csv(PROCESSED_PATH)

# 1.0. Define Mediation Sample & Center Variables (Imputed)

Uses imputed variables throughout

In [ ]:
# Focal variables (imputed)
h4_vars_base = [
    # Outcome
    "UTAUT_AI_mean_imputed",
    # Predictor (X)
    "TENS_Life_mean_imputed",
    # Mediator (M)
    "ET_mean_imputed",
    # Covariates
    "age_imputed", "gender", "Country", "role_label",
    "PHQ5_mean_imputed", "SSRPH_mean_imputed", "GAAIS_mean_imputed",
]
missing_cols = [c for c in h4_vars_base if c not in processed.columns]

## 1.1. Center key continuous predictors for interpretability

In [ ]:
for col in ["TENS_Life_mean_imputed", "ET_mean_imputed", "age_imputed"]:
    mean_val = processed[col].mean()
    processed[f"{col}_c"] = processed[col] - mean_val
    print(f"{col} mean for centering: {mean_val:.3f}")

In [ ]:
# Build full mediation dataframe (including centered vars)
h4_vars_full = h4_vars_base + [
    "TENS_Life_mean_imputed_c",
    "ET_mean_imputed_c",
    "age_imputed_c",
]

h4_df_full = processed[h4_vars_full].dropna().copy()
print(f"H4 full-control mediation sample (listwise): N = {len(h4_df_full)}")

In [ ]:
# Descriptives for mediation-relevant variables
print("Descriptives for mediation variables (imputed, centered where applicable):")
display(
    h4_df_full[
        ["TENS_Life_mean_imputed_c", "ET_mean_imputed_c", "UTAUT_AI_mean_imputed"]
    ].describe().T
)

In [ ]:
print("Pearson correlations among X (TENS), M (ET), Y (UTAUT_AI):")
display(
    h4_df_full[
        ["TENS_Life_mean_imputed_c", "ET_mean_imputed_c", "UTAUT_AI_mean_imputed"]
    ].corr()
)

# 2.0. Model Specification (Full-Control)

We first estimate the mediation pathway **controlling for the same covariates used in earlier hypothesis tests (H1–H3)** to keep the adjustment set consistent:

- **Covariates:** age_c, gender, PHQ5_mean (depressive symptoms),
  SSRPH_mean (self-stigma), GAAIS_mean (general AI attitudes),
  Country (China vs USA), and role_label (client vs therapist vs unknown).

Thus:

- **Path a model (X → M):**

  `ET_mean_imputed_c ~ TENS_Life_mean_imputed_c + age_imputed_c + C(gender) + PHQ5_mean_imputed + SSRPH_mean_imputed + GAAIS_mean_imputed + C(Country) + C(role_label)`

- **Path b + c′ model (M and X → Y):**

  `UTAUT_AI_mean_imputed ~ TENS_Life_mean_imputed_c + ET_mean_imputed_c + age_imputed_c + C(gender) + PHQ5_mean_imputed + SSRPH_mean_imputed + GAAIS_mean_imputed + C(Country) + C(role_label)`

We estimate these models with OLS using listwise deletion on the full set of variables.

## 2.1. Path a: X → M (TENS → ET), adjusting for full covariates

In [ ]:
print(f"H4 full-control sample size (listwise): N = {len(h4_df_full)}")
a_model_full = smf.ols(
    formula=(
        "ET_mean_imputed_c ~ "
        "TENS_Life_mean_imputed_c "
        "+ age_imputed_c + C(gender) "
        "+ PHQ5_mean_imputed + SSRPH_mean_imputed + GAAIS_mean_imputed "
        "+ C(Country) + C(role_label)"
    ),
    data=h4_df_full,
).fit()

print("H4 Path a (TENS_Life_mean_imputed_c → ET_mean_imputed_c; full-control):")
display(a_model_full.summary().tables[1])

In [ ]:
print(f"R² (Path a, full-control) = {a_model_full.rsquared:.3f}")

TENS → ET: b ≈ −0.025, p ≈ .13 → not significant

Strong positive associations of ET with:
- PHQ5 (b ≈ 0.21, p < .001)
- SSRPH (b ≈ 0.28, p < .001)
- GAAIS (b ≈ 0.12, p < .001)

R² ≈ .25

Once we partial out symptoms, stigma, and general AI attitudes, there’s no clean residual link from need satisfaction to epistemic trust.

## 2.2. Paths b and c': M and X → Y, adjusting for same covariates

In [ ]:
b_model_full = smf.ols(
    formula=(
        "UTAUT_AI_mean_imputed ~ "
        "TENS_Life_mean_imputed_c + ET_mean_imputed_c "
        "+ age_imputed_c + C(gender) "
        "+ PHQ5_mean_imputed + SSRPH_mean_imputed + GAAIS_mean_imputed "
        "+ C(Country) + C(role_label)"
    ),
    data=h4_df_full,
).fit()

print("H4 Paths b + c' (ET_mean_imputed_c and TENS_Life_mean_imputed_c → UTAUT_AI_mean_imputed; full-control):")
display(b_model_full.summary().tables[1])

In [ ]:
print(f"R² (Path b/c', full-control) = {b_model_full.rsquared:.3f}")

ET → UTAUT (b-path): b ≈ 0.205, p < .001 → clear positive effect.

TENS → UTAUT (c′): b ≈ 0.094, p < .001 → small but solid positive direct effect.

Strong positive effect of:

- Country (USA vs China) ≈ 1.09, p < .001
- GAAIS (b ≈ 0.24, p < .001)
- R² ≈ .29

Indirect (full-control)
- a ≈ −0.025, b ≈ 0.205 → a×b ≈ −0.0051 → tiny, likely not practically meaningful.

Conclusion for full-control model

- Mediation is not supported in the classic “needs → trust → acceptance” sense.
- ET is a strong independent predictor of UTAUT.
- TENS has its own direct positive effect on UTAUT (controlling for ET, symptoms, stigma, GAAIS, Country, role).

# 3.0. Parsimonious Mediation Model (Demographics + Country Only)

### 3.1. Parsimonious mediation model

In [ ]:
h4_vars_simple = [
    "UTAUT_AI_mean_imputed",
    "TENS_Life_mean_imputed_c",
    "ET_mean_imputed_c",
    "age_imputed_c",
    "gender",
    "Country",
]

h4_df_simple = processed[h4_vars_simple].dropna().copy()
print(f"H4 parsimonious mediation sample (listwise): N = {len(h4_df_simple)}")

### 3.2.  Path a (parsimonious): TENS → ET with minimal covariates

ET_mean_imputed_c ~ TENS_Life_mean_imputed_c + age_imputed_c + C(gender) + C(Country)

- TENS → ET: b ≈ −0.0579, p ≈ .001 → significant negative association.
- R² ≈ .10

In [ ]:
a_model_simple = smf.ols(
    formula=(
        "ET_mean_imputed_c ~ "
        "TENS_Life_mean_imputed_c + age_imputed_c + C(gender) + C(Country)"
    ),
    data=h4_df_simple,
).fit()

print("H4 Path a (TENS_Life_mean_imputed_c → ET_mean_imputed_c; parsimonious):")
display(a_model_simple.summary().tables[1])

In [ ]:
print(f"R² (Path a, parsimonious) = {a_model_simple.rsquared:.3f}")

## 3.3. Path b + c': ET + TENS → UTAUT (parsimonious)

UTAUT_AI_mean_imputed ~ TENS_Life_mean_imputed_c + ET_mean_imputed_c + age_imputed_c + C(gender) + C(Country)

- ET → UTAUT: b ≈ 0.265, p < .001.
- TENS → UTAUT: b ≈ 0.128, p < .001.
- Country huge effect again (USA > China).

R² ≈ .27.

In [ ]:
b_model_simple = smf.ols(
    formula=(
        "UTAUT_AI_mean_imputed ~ "
        "TENS_Life_mean_imputed_c + ET_mean_imputed_c "
        "+ age_imputed_c + C(gender) + C(Country)"
    ),
    data=h4_df_simple,
).fit()

print("H4 Paths b + c' (ET_mean_imputed_c and TENS_Life_mean_imputed_c → UTAUT_AI_mean_imputed; parsimonious):")
display(b_model_simple.summary().tables[1])

In [ ]:
print(f"R² (Path b/c', parsimonious) = {b_model_simple.rsquared:.3f}")

### 3.4. Point estimate of indirect effect

a ≈ −0.0579, b ≈ 0.2652 → a×b ≈ −0.0154.

In [ ]:
a_coef_simple = a_model_simple.params.get("TENS_Life_mean_imputed_c", np.nan)
b_coef_simple = b_model_simple.params.get("ET_mean_imputed_c", np.nan)
cprime_simple = b_model_simple.params.get("TENS_Life_mean_imputed_c", np.nan)
indirect_simple_ab = a_coef_simple * b_coef_simple

In [ ]:
print("Key parsimonious path estimates (using imputed variables):")
print(f"  a (TENS → ET | demo + Country)        = {a_coef_simple:.4f}")
print(f"  b (ET → UTAUT | TENS, demo + Country) = {b_coef_simple:.4f}")
print(f"  c' (TENS → UTAUT | ET, demo + Country)= {cprime_simple:.4f}")
print(f"  a × b (indirect, parsimonious)        = {indirect_simple_ab:.4f}")

## 3.5. Multicollinearity & diagnostics

Full-control model b-path includes: TENS, ET, PHQ5, SSRPH, GAAIS, age, gender, Country, role.

Everything is still VIF < ~4, we can comfortably say multicollinearity is not severe


In [ ]:
# VIFs for parsimonious H4 b-model
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = b_model_simple.model.exog
names = b_model_simple.model.exog_names

vif_rows = []
for i, name in enumerate(names):
    if name == "Intercept":
        continue
    vif_rows.append({
        "Predictor": name,
        "VIF": variance_inflation_factor(X, i)
    })

vif_h4_b = pd.DataFrame(vif_rows).sort_values("VIF", ascending=False)
display(vif_h4_b)

# 4.0. Bootstrap for Indirect Effect

## 4.1. Bootstrap for indirect effect a × b

Bootstrap:

mean ≈ −0.0155

95% CI ≈ [−0.0290, −0.0031] → CI excludes 0, but effect is small.

In [ ]:
n_boot = 5000
boot_indirect = []
n = len(h4_df_simple)

rng = np.random.default_rng(seed=42)  # reproducible

for _ in range(n_boot):
    sample_idx = rng.integers(0, n, size=n)
    sample = h4_df_simple.iloc[sample_idx]

    # Path a in bootstrap sample
    a_m = smf.ols(
        formula=(
            "ET_mean_imputed_c ~ "
            "TENS_Life_mean_imputed_c + age_imputed_c + C(gender) + C(Country)"
        ),
        data=sample,
    ).fit()

    # Path b + c' in bootstrap sample
    b_m = smf.ols(
        formula=(
            "UTAUT_AI_mean_imputed ~ "
            "TENS_Life_mean_imputed_c + ET_mean_imputed_c "
            "+ age_imputed_c + C(gender) + C(Country)"
        ),
        data=sample,
    ).fit()

    a_b = a_m.params.get("TENS_Life_mean_imputed_c", np.nan)
    b_b = b_m.params.get("ET_mean_imputed_c", np.nan)

    if not np.isnan(a_b) and not np.isnan(b_b):
        boot_indirect.append(a_b * b_b)

boot_indirect = np.array(boot_indirect)
indirect_mean = np.mean(boot_indirect)
ci_lower, ci_upper = np.percentile(boot_indirect, [2.5, 97.5])

In [ ]:
print("Bootstrapped indirect effect (parsimonious a × b):")
print(f"  Mean indirect effect = {indirect_mean:.4f}")
print(f"  95% CI               = [{ci_lower:.4f}, {ci_upper:.4f}]")
print(f"  Successful bootstraps: {len(boot_indirect)} / {n_boot}")

Clear suppression / inconsistent mediation:

- TENS directly increases UTAUT.
- TENS is (slightly) associated with lower ET.
- ET strongly increases UTAUT.

Therefore the indirect effect is small and negative, slightly reducing the overall positive relationship between TENS and UTAUT.

In [ ]:
# Visualize distribution of bootstrap indirect effects
plt.figure(figsize=(7, 4))
sns.histplot(boot_indirect, bins=40, kde=True)
plt.axvline(indirect_mean, linestyle="--", linewidth=1)
plt.title("H4: Bootstrap distribution of indirect effect (a × b)")
plt.xlabel("Indirect effect (TENS → ET → UTAUT)")
plt.tight_layout()
plt.show()

# Narrative Summary

In [ ]:
summary_rows = []

# Safely extract coefficients from full-control models
if "a_model_full" in globals() and "b_model_full" in globals():
    a_coef_full = a_model_full.params.get("TENS_Life_mean_imputed_c", np.nan)
    b_coef_full = b_model_full.params.get("ET_mean_imputed_c", np.nan)
    cprime_full = b_model_full.params.get("TENS_Life_mean_imputed_c", np.nan)
    indirect_full_ab = a_coef_full * b_coef_full

    summary_rows.append({
        "Model": "Full-control",
        "a (TENS→ET)": a_coef_full,
        "b (ET→UTAUT)": b_coef_full,
        "c' (TENS→UTAUT)": cprime_full,
        "a×b (indirect)": indirect_full_ab,
        "Boot a×b mean": np.nan,
        "Boot a×b CI_low": np.nan,
        "Boot a×b CI_high": np.nan,
    })
else:
    print("Warning: a_model_full / b_model_full not found; skipping full-control row.")

# Safely extract coefficients from parsimonious models
if "a_model_simple" in globals() and "b_model_simple" in globals():
    a_coef_simple = a_model_simple.params.get("TENS_Life_mean_imputed_c", np.nan)
    b_coef_simple = b_model_simple.params.get("ET_mean_imputed_c", np.nan)
    cprime_simple = b_model_simple.params.get("TENS_Life_mean_imputed_c", np.nan)
    indirect_simple_ab = a_coef_simple * b_coef_simple

    summary_rows.append({
        "Model": "Parsimonious",
        "a (TENS→ET)": a_coef_simple,
        "b (ET→UTAUT)": b_coef_simple,
        "c' (TENS→UTAUT)": cprime_simple,
        "a×b (indirect)": indirect_simple_ab,
        "Boot a×b mean": indirect_mean,
            # indirect_mean, ci_lower, ci_upper should come from the bootstrap cell
        "Boot a×b CI_low": ci_lower,
        "Boot a×b CI_high": ci_upper,
    })
else:
    print("Warning: a_model_simple / b_model_simple not found; skipping parsimonious row.")

h4_summary_table = pd.DataFrame(summary_rows)
print("H4 mediation summary (imputed, processed dataset):")
display(h4_summary_table)